In [1]:
from z3 import *

In [2]:
# U iskaznoj logici zapisati uslov da je 4-bitna reprezentacija broja
# palindrom, ali da nisu svi bitovi isti.

A,B,C,D = Bools('A B C D')

s = Solver()
s.add(A == D, B == C, Not(And(A == B, B == C, C == D)))

while s.check() == sat:
    print(s.model())
    # Ispis svih modela. U ogranicenje dodamo negaciju prethodnog resenja.
    # s.model()[A] -> vrednost promenljive A u poslednjem modelu koji zadovoljava ogranicenja
    s.add(Not(And(A == s.model()[A], B == s.model()[B], C == s.model()[C], D == s.model()[D])))

[A = False, D = False, B = True, C = True]
[A = True, D = True, B = False, C = False]


In [3]:
x, y, z = Reals('x y z')

s = Solver()
s.add(
    x + 5*y - 3*z == 4, # 1/2 +  7/2 = 8/2 = 4
    -x + y + z == 3,
    2*x + y - z == 1
)

if s.check() == sat:
    print(s.model())

[z = 7/4, y = 7/4, x = 1/2]


In [4]:
# Bool, Real, Int, BitVec, Array, Index, String...
# Function

# Sort == Domen
B = BoolSort()
Z = IntSort()
# f: B -> Z
f = Function('f', B, Z)
# g: Z -> B
g = Function('g', Z, B)
a = Bool('a')
solve(g(1 + f(a)))

[a = False, f = [else -> 0], g = [else -> True]]


In [5]:
x, y = Ints('x y')
# Za svako y, ako je y <= 0, onda je x < y.  i y == x + 1.
solve([y == x + 1, ForAll([y], Implies(y <= 0, Exists([x], x < y)))])

[x = 0, y = 1]


In [6]:
# Dve nemimoilazne prave se seku ili su paralelne.
# Prave koje se seku leže u istoj ravni.
# Prave koje su paralelene leže u istoj ravni.
# Dve nemimoilazne prave leže u istoj ravni.

# m(X,Y) - X i Y su nemimoilazne: m: PxP -> B
# s(X,Y) - X i Y se seku: s: PxP -> B
# p(X,Y) - X i Y su paralelne: p: PxP -> B
# r(X,Y) - X i Y leze u istoj ravni: r: PxP -> B

B = BoolSort()
P = DeclareSort('Prave')
m = Function('m', P, P, B)
s = Function('s', P, P, B)
p = Function('p', P, P, B)
r = Function('r', P, P, B)

x, y = Consts('x y', P)

solver = Solver()
axioms = [
    ForAll([x,y], Implies(m(x,y), Or(s(x,y), p(x,y)))),
    ForAll([x,y], Implies(s(x,y), r(x,y))),
    ForAll([x,y], Implies(p(x,y), r(x,y)))
]
conjecture = ForAll([x,y], Implies(m(x,y), r(x,y)))

solver.add(conjecture)

print(solver.check(axioms))

sat


In [7]:
# Svaka dva brata imaju zajedničkog roditelja.
# Roditelj je stariji od deteta.
# Postoje braća.
# Ni jedna osoba nije starija od druge.

# b: OxO -> B

B = BoolSort()
O = DeclareSort('Osoba')

b = Function('braca', O, O, B)
r = Function('roditelj', O, O, B)
s = Function('stariji', O, O, B)

x,y,z = Consts('x y z', O)

solver = Solver()
axioms = [
    ForAll([x,y], Exists([z], Implies(b(x,y), And(r(z,x), r(z,y))))),
    ForAll([x,y], Implies(r(x,y), s(x,y))),
    Exists([x,y], b(x,y))
]
conjecture = ForAll([x,y], Not(s(x,y)))

solver.add(conjecture)
solver.check(axioms)

unsat

In [95]:
#Bool p_i_j - Na polju (i,j) se nalazi dama
#Int Q_{i} -> Broj kolone u kojoj se nalazi dama u i-tom redu
# Q_4 = 3 -> U cetvrtom redu, dama se nalazi na trecoj koloni

# Q_3 = 2, Q_4 = 2

n = 8
Q = [Int(f'Q_{i}') for i in range(n)]

val_c = [And(0 <= q, q < n) for q in Q]
col_c = [Distinct(Q)]
diag_c = [
    And(Q[i] - Q[j] != i - j, Q[i] - Q[j] != j - i) 
    for i in range(n) for j in range(i) if i != j
]
setup_c = [If(i == 7, Q[7] == 2, True) for i in range(n)]

n_queens = val_c + col_c + diag_c + setup_c

solve(n_queens)

[Q_1 = 3,
 Q_0 = 5,
 Q_5 = 6,
 Q_6 = 0,
 Q_3 = 7,
 Q_2 = 1,
 Q_4 = 4,
 Q_7 = 2]


In [91]:
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ] 
      for i in range(9) ]


cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9) 
             for i in range(9) for j in range(9) ]


rows_c   = [ Distinct(X[i]) for i in range(9) ]


cols_c   = [ Distinct([ X[i][j] for i in range(9) ]) 
             for j in range(9) ]

sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j] 
                        for i in range(3) for j in range(3) ]) 
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

instance_c = [ If(instance[i][j] != 0, 
                  X[i][j] == instance[i][j], 
                  True) 
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ] 
          for i in range(9) ]
    print_matrix(r)
else:
    print("failed to solve")

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]
